## Note
series, works and author are whole files, but genres, review and books are only taking the first 100000 rows. 
genre dataset has 2360655 rows in total;
25 G is not enough for reading the books file

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import json
import numpy as np
import pandas as pd
import ast

In [0]:
def preprocess(df):
  threshold=0.1
  df.replace(["", 'NaN'], np.nan, inplace = True)
  #remove columns with too many nan
  df=df[df.columns[df.isnull().mean()<threshold]]
  #fill nan with mode
  for col in df:
    if (df[col].dtype!=int):
      continue
    df[col] = df[col].fillna(df[col].mode())
  
    '''
  #numerical outliers
  #not working
  for col in df.select_dtypes([np.number]):
    print('working')
    Q1 = col.quantile(0.02)
    Q3 = col.quantile(0.98)
    IQR = Q3 - Q1
    df= df[~((col< (Q1 - 1.5 * IQR)) |(col> (Q3 + 1.5 * IQR))).any(axis=1)]
    '''
  #all 0 columns
  df = df[(df.T != 0).any()]
  df=pd.DataFrame(df)
  return df

In [4]:
genres=[]
count=0
with open ('/content/drive/My Drive/good_reads/goodreads_book_genres_initial.json/goodreads_book_genres_initial.json') as f:
  for line in f:
    line=json.loads(line)
    if line['genres']!='{}':
      genres.append(line)
      count+=1
    if count>100000:
      break
genres=pd.DataFrame(genres)    
genres.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [0]:
genres=preprocess(genres) #too large, ram not enough
genres.to_csv('/content/drive/My Drive/good_reads/genres.csv')

In [6]:
genres.shape

(100001, 2)

In [7]:
genres.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [0]:
book_genre=[]
for bid, gen in zip(genres['book_id'],genres['genres']):
    #print(gen)
    if gen !='{}' and gen!= {}:
      book_genre.append([bid,max(gen, key=gen.get)])
    


In [11]:
book_genre=pd.DataFrame(book_genre)
book_genre.head()
book_genre.shape

(83005, 2)

In [0]:
book_genre.to_csv('/content/drive/My Drive/good_reads/book_genre.csv')

In [13]:
books=[]
count=0
with open ('/content/drive/My Drive/good_reads/goodreads_books.json', encoding='utf-8') as f:
  for line in f:
    books.append(json.loads(line))
    count+=1
    if count>1000000:
      break
books=pd.DataFrame(books)    
books.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,0312853122,1,[],US,,"[{'count': '3', 'name': 'to-read'}, {'count': ...",,false,4.00,,[],,Paperback,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",St. Martin's Press,256,1,9780312853129,9,,1984,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3,5400751,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film
1,0743509986,6,[],US,,"[{'count': '2634', 'name': 'to-read'}, {'count...",,false,3.23,B000FC0PBC,"[8709549, 17074050, 28937, 158816, 228563, 112...","Anita Diamant's international bestseller ""The ...",Audio CD,https://www.goodreads.com/book/show/1333909.Go...,"[{'author_id': '626222', 'role': ''}]",Simon & Schuster Audio,,1,9780743509985,10,Abridged,2001,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10,1323437,Good Harbor,Good Harbor
2,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600,,,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",,false,3.49,B002ENBLOK,"[6604176, 6054190, 2285777, 82641, 7569453, 70...",Addie Downs and Valerie Adler were eight when ...,Hardcover,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",Atria Books,368,14,9780743294294,7,,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever
4,0850308712,5,[],US,,"[{'count': '32', 'name': 'to-read'}, {'count':...",,false,3.40,,[],,,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",,,,9780850308716,,,,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15,278577,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...


In [0]:
books=preprocess(books)


In [0]:
books=books.loc[books['title'].str.contains(r'[^\x00-\x7F]+') == False]

In [16]:
books.head()
books.shape

(895130, 16)

In [0]:
authors=books[['book_id' ,'authors']]


In [0]:
import ast


In [0]:

book_author=[]
for book_id, author in zip(authors['book_id'],authors['authors']):
  contents=eval(str(author))
  author_id=[]
  for i in contents:
    book_author.append([book_id,i['author_id']])



In [23]:
authorof=pd.DataFrame(book_author)
authorof.head()

,0,1
0,5333265,604031
1,1333909,626222
2,7327624,10333
3,6066819,9212
4,287140,149918


In [24]:
authorof.describe()

,0,1
count,1243415,1243415
unique,894920,424531
top,12771197,123715
freq,80,1571


In [25]:
authorof.shape

(1243415, 2)

In [26]:
authorof.head()

,0,1
0,5333265,604031
1,1333909,626222
2,7327624,10333
3,6066819,9212
4,287140,149918


In [0]:
authorof=pd.DataFrame(book_author)

In [0]:
authorof.to_csv('/content/drive/My Drive/good_reads/authorof.csv')

In [0]:
books.to_csv('/content/drive/My Drive/good_reads/books.csv')